In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import time
import glob
import torch
import tensorflow as tf
import tensorflow_hub as hub
import onnxruntime
from my_utils import patch_extractor, draw_connections, distortion_free_resize
from my_utils import get_detector
from model import transformer
import concurrent.futures         # Learn about this

In [2]:
model_dir = "weights/best.pt"
model = torch.hub.load("ultralytics/yolov5", "custom", path = model_dir, force_reload=False)

Using cache found in C:\Users\DELL Inspiron/.cache\torch\hub\ultralytics_yolov5_master
requirements: YOLOv5 requirements "gitpython" "setuptools>=65.5.1" "wheel>=0.38.0" not found, attempting AutoUpdate...
requirements:  Command 'pip install "gitpython" "setuptools>=65.5.1" "wheel>=0.38.0"  ' returned non-zero exit status 101.
YOLOv5  2023-2-19 Python-3.7.15 torch-1.13.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients
Adding AutoShape... 


In [3]:
sess = onnxruntime.InferenceSession("movenet_thunder.onnx")
input_name = sess.get_inputs()[0].name
input_shape = sess.get_inputs()[0].shape

In [4]:
test_video_dir = "C:/Users/DELL Inspiron/Desktop/Computer Vision Projects/Thesis_data/IMG_2573.MOV"

In [6]:
cap = cv2.VideoCapture(test_video_dir)  # 0.2
ret, frame = cap.read()

def process_patch(patch):
    cropped_img, left, right, top, bot = distortion_free_resize(patch, (256, 256))
    kpts = sess.run([], {input_name: cropped_img.numpy()[np.newaxis, ...]})
    #cropped_img = cropped_img.numpy()
    #draw_connections(cropped_img, kpts, 0.15)
    return kpts

while True:
    start = time.time()
    ret, frame = cap.read()

    if ret:
        processed_frame = frame.copy()[:, :, [2, 1, 0]]
        preds = model(processed_frame).xyxyn[0]
        
        patches = []
        
        for pred in preds:
            x1, y1, x2, y2 = patch_extractor(pred)
            patch = processed_frame[y1:y2, x1:x2, :]
            patches.append(patch)
            
            
        with concurrent.futures.ThreadPoolExecutor(max_workers = 20) as executor:
            results = list(executor.map(process_patch, patches))
        
        
#         for i in range(len(coords)):
#             img = tf.cast(tf.image.resize(results[i]["cropped_img"], size=(coords[i][3] - coords[i][1], coords[i][2] - coords[i][0])),
#                     dtype=tf.int32).numpy()
#             processed_frame[coords[i][1]:coords[i][3], coords[i][0]:coords[i][2], :] = img
        
        
    #cv2.imshow("Visualization", processed_frame[:, :, [2, 1, 0]])
    
    print(time.time() - start)
    if cv2.waitKey(10) == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


0.6376018524169922
0.6170721054077148
0.6029200553894043
0.6659915447235107
0.6359462738037109
0.6509091854095459
0.6527512073516846
0.6103568077087402
0.6521410942077637
0.7809398174285889
0.7332086563110352
0.6978330612182617
0.6350655555725098
0.6227824687957764
0.6394426822662354
0.6417241096496582
0.6986963748931885
0.7293076515197754
0.6640336513519287
0.654155969619751
0.6132676601409912
0.6157486438751221


KeyboardInterrupt: 